In [104]:
import pandas as pd
import numpy as np
import torch 
from tqdm import tqdm 
from sklearn.model_selection import train_test_split
import glob, os, pickle
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from torch import nn

In [102]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Create Train/Test/Validation split 
- First, considerably subset the data, from around 450k to about 100k. 
- It'll take around 7 minutes to load the patch_class

In [127]:
#write a different data loader class 
class Patch_Class():
    def __init__(self, csv_path, root_dir, transform=None):
        self.patch_frame = pd.read_csv(csv_path) #get the metadata 
        self.root_dir = root_dir
        self.transform = transform
        
        #we also need to build the patch dictionary, which maps sample_id to patch_id to status 
        self.patch_dict = {}
        self.build_dictionary()
        
        #here, we also need to load in all of the distinct np arrays for each directory
        self.data_dict = {}
        self.build_data()
        
    def build_data(self):
        #go through each sub dir in the main dir 
        for s_dir in tqdm(os.listdir(self.root_dir)):
            if s_dir != "metadata.csv":
                data = np.load(self.root_dir + s_dir +"/data.npy")
                self.data_dict[s_dir] = data #map the sample_id to the npy data 
                
    def build_dictionary(self):
        for s_dir in tqdm(os.listdir(self.root_dir)):
            #now, for each row, make the dictionary
            self.patch_dict[s_dir] = {}
        for id, group in tqdm(self.patch_frame.groupby("ID")):
            for idx, group2 in group.groupby("patch_index"):
                self.patch_dict[id][idx] = (group2["scc"] == True)
            
    def __len__(self):
        return len(self.patch_frame)

    def __getitem__(self, index):
        #1 is the file id
        sample_id = self.patch_frame.iloc[index, 1]
        patch_id = self.patch_frame.iloc[index, 8]
        #get the image as a numpy array 
        img = self.data_dict[sample_id][patch_id]
        #get y_label and one hot encode it
#         ohe = [0, 0]
        y_label = int(list(self.patch_dict[sample_id][patch_id])[0])
#         ohe[y_label] = 1
        y_label = torch.tensor(y_label)

        if self.transform: 
            img = self.transform(img)
        return (img, y_label)

In [128]:
#transform the function according to the pytorch docs
from torchvision import transforms
from PIL import Image
# img_size = 224
preprocess = transforms.Compose([
#     transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [129]:
path = "/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Gokul_Srinivasan/SCC-Tumor-Detection/Gokul_files/data/metadata.csv"

root_dir = "/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Gokul_Srinivasan/SCC-Tumor-Detection/Gokul_files/data/"

patches = Patch_Class(path, root_dir, transform = preprocess)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 31/31 [00:29<00:00,  1.07it/s]


In [130]:
patches.__getitem__(231)

(tensor([[[ 2.0263,  2.0263,  2.0263,  ...,  0.5878,  0.4851,  0.2796],
          [ 2.0263,  2.0263,  2.0263,  ...,  0.0056, -0.0458, -0.1828],
          [ 2.0263,  2.0263,  2.0263,  ..., -0.4226, -0.4397, -0.4568],
          ...,
          [ 2.0092,  2.1462,  1.6838,  ...,  2.0434,  1.3755,  1.0331],
          [ 2.0092,  2.1633,  1.5468,  ...,  1.7180,  1.2043,  1.0673],
          [ 1.8208,  2.2318,  1.8893,  ...,  1.1187,  0.8789,  0.9474]],
 
         [[ 2.2010,  2.2010,  2.2010,  ...,  0.1527,  0.0476, -0.1625],
          [ 2.2010,  2.2010,  2.2010,  ..., -0.4076, -0.5126, -0.6001],
          [ 2.2010,  2.2010,  2.2010,  ..., -0.8102, -0.8627, -0.8452],
          ...,
          [ 2.1835,  2.3235,  1.8508,  ...,  1.4307,  0.6954,  0.2927],
          [ 2.1835,  2.3410,  1.7108,  ...,  1.0630,  0.4503,  0.2927],
          [ 1.9909,  2.4111,  2.0609,  ...,  0.4328,  0.1176,  0.1702]],
 
         [[ 2.4134,  2.4134,  2.4134,  ...,  1.3677,  1.2631,  1.0539],
          [ 2.4134,  2.4134,

# Create the Dataloader

In [131]:
#first trim the OG dataset down further, so we randomly retain a 10th of the data. 
dataset, discard = torch.utils.data.random_split(patches, [int(len(patches)*.10), int(len(patches)*.9)+1])
len(dataset)

43851

In [132]:
#create train/test
train_set, test_set = torch.utils.data.random_split(dataset, [int(len(dataset)*.75), int(len(dataset)*.25)+1])
#create train/validation
train_set, val_set = torch.utils.data.random_split(train_set, [int(len(train_set)*.9), int(len(train_set)*.1)+1])
print(len(train_set), len(val_set), len(test_set))

29599 3289 10963


In [133]:
batch_size = 32 

train_loader = DataLoader(dataset = train_set, batch_size = batch_size, shuffle=True)
val_loader = DataLoader(dataset = train_set, batch_size = batch_size, shuffle=True)
test_loader = DataLoader(dataset = test_set, batch_size = batch_size, shuffle=True)

# Load Model
- Also change the architecture slightly 

In [134]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True)

Using cache found in /dartfs-hpc/rc/home/9/f003xr9/.cache/torch/hub/pytorch_vision_v0.10.0


In [135]:
#we can also set the first, say, n layers to be frozen, and leave the remaining layers unfrozen, as follows 
thresh = 6
ct = 0
#here we freeze up to and including the 6th layer
for child in model.children():
    if ct <= thresh:
        for param in child.parameters():
            param.requires_grad = False
        print(child, ct)
        ct += 1

Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False) 0
BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True) 1
ReLU(inplace=True) 2
MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False) 3
Sequential(
  (0): Bottleneck(
    (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (downsample): Sequential(
      (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine

In [136]:
#change the model architecture a bit
model.fc = nn.Sequential(nn.Linear(2048, 100), 
                         nn.ReLU(), 
                         nn.Dropout(p=.5), 
                         nn.Linear(100,2))
model

model.train()
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

# Model Training 

In [137]:
# Check accuracy on training to see how good our model is
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()
    correct = {0:0, 1:0, 2:0, 3:0, 4:0, 5:0, 6:0}
    total = {0:0, 1:0, 2:0, 3:0, 4:0, 5:0, 6:0}
    with torch.no_grad():
        for x, y, name in tqdm(loader):
            x = x.to(device=device)
            y = y.to(device=device)

            scores = model(x)
            _, predictions = scores.max(1)
            for i,j in zip(predictions, y):
                if i.item() == j.item():
                    correct[i.item()] +=1
                total[j.item()] += 1
                num_correct += (predictions == y).sum()
                num_samples += predictions.size(0)

        print(
              f"Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}"
          )
        acc = num_correct/num_samples
        #find the accuracies for each class 
        return acc, correct, total

    model.train()

In [125]:
#hyperparams
learning_rate = 1e-3
num_epochs =10 #20 works well - it seems as tho it is a local min 

Some notes
1. Might need to figure out another loss that works better with one hot encoding 
2. Also might need to figure out how to calc AUC-ROC 

In [139]:
# Loss and optimizer
import torch.optim as optim  # For all Optimization algorithms, SGD, Adam, etc.


criterion = nn.CrossEntropyLoss()
# criterion = tgm.losses.FocalLoss(alpha=0.5, gamma=2.0, reduction='mean') #experimenting with focal loss 
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=.1, patience=5, verbose=True)

#arrays to track the training loss and validation loss 
training_loss = []
validation_acc = []

# Train Network
for epoch in range(num_epochs):
    losses = []
    num_correct = 0
    num_samples = 0
    #train part 
    for batch_idx, (data, targets) in tqdm(enumerate(train_loader)):
        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)
        # forward
        scores = model(data)
        loss = criterion(scores, targets)
        # print("Batch: %d. Loss: %f" %(batch_idx, loss))

        losses.append(loss.item())

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()
    mean_loss = sum(losses)/len(losses)
    training_loss.append(mean_loss)
    scheduler.step(mean_loss)

    print(f"Cost at epoch {epoch} is {sum(losses)/len(losses)}")
    
    #model in test mode 
    model.eval()
    with torch.no_grad():
        acc = 0
        for x, y in tqdm(val_loader):
            x = x.to(device=device)
            y = y.to(device=device)

            scores = model(x)
            #find the test loss
            loss = criterion(scores, y)


            #find the test accuracy 
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)
        #calc total acc here 
        acc = (num_correct/num_samples).item()
        print(acc)
        validation_acc.append(acc)
    #put the model back in train mode
    model.train()

925it [00:56, 16.32it/s]


Cost at epoch 0 is 0.31966916019046626


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 925/925 [00:38<00:00, 24.20it/s]


0.8937802314758301


925it [00:58, 15.71it/s]


...

0.9951688051223755


925it [00:58, 15.73it/s]


Cost at epoch 9 is 0.054398015554855905


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 925/925 [00:38<00:00, 24.24it/s]

0.9966891407966614
